* [PyTorch Tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)
* [Do not need softmax](https://stackoverflow.com/questions/55675345/should-i-use-softmax-as-output-when-using-cross-entropy-loss-in-pytorch)
* I learned how to implement LeNet5
* I learned how to train net on GPU

In [32]:
# I would like to plot my  diagram inside the cell.
%matplotlib inline

## My Customized Dataset Class
In order to implement this section, I read two very good references:
* [WRITING CUSTOM DATASETS, DATALOADERS AND TRANSFORMS](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)
* [utkuozbulak/pytorch-custom-dataset-examples](https://github.com/utkuozbulak/pytorch-custom-dataset-examples)

In [33]:
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import numpy as np
import pandas as pd
import cv2 as cv

class AnimalDataset(Dataset):
    def __init__(self, root_path, subset, height, width):
        self.root_path = root_path
        
        # Need it in __getitem__() to transform numpy array to tensor.
        self.to_tensor = transforms.ToTensor()
        
        # Read CSV Metadata and convert it to numpy format
        subset_txt = pd.read_csv(root_path + subset, sep=" ")
        self.np_subset_txt = np.array(subset_txt)
        
        # Need it in __len__()
        self.count = len(self.np_subset_txt)
        
        # Need it in __getitem__() to resize image to desired size
        self.height = height
        self.width = width
    
    def __getitem__(self, index):
        # Get Label
        label = int(self.np_subset_txt[index][1])
        
        # Get Image
        img = cv.imread(self.root_path + self.np_subset_txt[index][0])
        img_resize = cv.resize(img, (self.height, self.width))/255.0
        img_resize = img_resize.reshape(-1, self.height, self.width) # to make channel first
        img_resize_tensor = self.to_tensor(img_resize)
        return (img_resize, label)
    
    def __len__(self):
        return self.count

In [ ]:
AC_root_path="C:/Users/rathe/Desktop/Dataset/images/"
root_path = "C:/Users/USER/Desktop/Projects/Github_Repo/AI/DeepLearning/__HW1_DATA/"
height = 30
width = 30

Train_Dataset = AnimalDataset(root_path = root_path, subset = "train.txt", height = height, width = width)
Test_Dataset = AnimalDataset(root_path = root_path, subset = "test.txt", height = height, width = width)
Val_Dataset = AnimalDataset(root_path = root_path, subset = "val.txt", height = height, width = width)

## Define My DataLoader r.s.t. AnimalDataset

In [35]:
import torch
batch_size = 50
shuffle = True

Train_DataLoader = torch.utils.data.DataLoader(dataset=Train_Dataset, batch_size = batch_size, shuffle=shuffle)
Test_DataLoader = torch.utils.data.DataLoader(dataset=Test_Dataset, batch_size = batch_size, shuffle=shuffle)
Val_DataLoader = torch.utils.data.DataLoader(dataset=Val_Dataset, batch_size = batch_size, shuffle=shuffle)

## Define a Convolutional Neural Network(Use GPU)

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Use GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 50)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        #x = F.softmax(x)
        return x
    
lenet = LeNet()
lenet.to(device)

cuda:0


LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=50, bias=True)
)

## Define a loss function and Set up Hyperparameters

In [37]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(lenet.parameters(), lr=1e-3)

## Train the network on the training data
# How to use validation dataset
# Try to implement it as a function to call

In [ ]:
from tqdm import tqdm
import time

tic = time.time()
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(Train_DataLoader, 0):
        inputs, labels = data[0].to(device), data[1].to(device) # GPU
        #inputs, labels = data # CPU
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = lenet(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            correct = 0
            total = 0
            # Train
            with torch.no_grad(): # since we're not training, we don't need to calculate the gradients for our outputs
                for datum in tqdm(Train_DataLoader):
                    imgs, labs = datum[0].to(device), datum[1].to(device)
                    # calculate outputs by running images through the network 
                    outputs = lenet(imgs.float())
                    # the class with the highest energy is what we choose as prediction
                    _, preds = torch.max(outputs.data, 1)
                    total += labs.size(0)
                    correct += (preds == labs).sum().item()

            print('Accuracy of the network on the train images: %d %%' % (100 * correct / total))
            # Val
            with torch.no_grad(): # since we're not training, we don't need to calculate the gradients for our outputs
                for datum in tqdm(Val_DataLoader):
                    imgs, labs = datum[0].to(device), datum[1].to(device)
                    # calculate outputs by running images through the network 
                    outputs = lenet(imgs.float())
                    # the class with the highest energy is what we choose as prediction
                    _, preds = torch.max(outputs.data, 1)
                    total += labs.size(0)
                    correct += (preds == labs).sum().item()

            print('Accuracy of the network on the val images: %d %%' % (100 * correct / total))

toc = time.time()
print(toc - tic)
print('Finished Training')

  0%|                                                                                 | 1/1267 [00:00<02:45,  7.64it/s]

[1,   100] loss: 3.746


100%|██████████████████████████████████████████████████████████████████████████████| 1267/1267 [02:38<00:00,  8.01it/s]


Accuracy of the network on the test images: 5 %


  0%|                                                                                 | 1/1267 [00:00<02:55,  7.19it/s]

...

100%|██████████████████████████████████████████████████████████████████████████████| 1267/1267 [02:31<00:00,  8.38it/s]


Accuracy of the network on the test images: 14 %


  0%|                                                                                 | 1/1267 [00:00<02:53,  7.30it/s]

[7,   500] loss: 3.265


 12%|█████████▌                                                                     | 154/1267 [00:19<02:29,  7.46it/s]

## Save the model for later usage

In [29]:
PATH = './LeNet.pth'
torch.save(net.state_dict(), PATH)

## Test the network on the test data

In [31]:
lenet = LeNet()
lenet.load_state_dict(torch.load(PATH))

correct = 0
total = 0

with torch.no_grad(): # since we're not training, we don't need to calculate the gradients for our outputs
    for data in Test_DataLoader:
        images, labels = data
        # calculate outputs by running images through the network 
        outputs = lenet(images.float())
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

Accuracy of the network on the test images: 2 %
